# Example of running docking and scoring of ensitrelvir affinities in coronavirus targets

In [14]:
# Load OpenEye license, required by the asapdiscovery package
!export OE_LICENSE=$HOME/Documents/.openeye/oe_license.txt

For this example, we will use a reference SARS-CoV-2 crystal in complex with ensitrelvir. The ligand will be docked into related coronavirus targets that infect humans. 

In [ ]:
ref_pdb = "input_files/8dz0.pdb"

## 1. Sequence alignment

In this example, we perform a sequence-based search starting from a pdb file of SARS-CoV-2 in complex with Ensitrelvir (8dz0). This is done with `asap-spectrum seq-alignment` (See [documentation](https://asapdiscovery.readthedocs.io/en/latest/index.html) for instructions on how to use).

The sequence search is done by invoking the NCBI BLAST server over the internet, and it will take longer everytime a job is placed for a given sequence. Therefore, we advise to run the sequence alignment job only once and save the BLAST output as a .xml file for future runs. Below we provide the command for an initial sequence alignment job, starting from a pdb file (uncomment to use).

An email address is required the Entrez database system, which is used for retrieving information on viral hosts (for the filtering step of the alignment process). 

In [ ]:
#!asap-spectrum seq-alignment \
#        -f {ref_pdb} \
#        -t "pdb" \
#        --output-dir "data/sars2_alignment" \
#        --sel-key "host: Homo sapiens OR organism: human" \
#        --multimer --n-chains 2 \
#        --color-seq-match --plot-width 900 \
#        --max-mismatches 2 \
#        --email "your_email.org" # !! Replace with your email that will be used for Entrez (viral host search)

By default, the results from the BLAST search above are saved as `results.xml`. We provide this file in the `data/` directory. **After replacing the email argument**, the run below will generate the necessary files for the rest of the pipeline, along with a html visualization of the alignment. 

In [ ]:
!asap-spectrum seq-alignment \
        -f "data/results.xml" \
        -t "pre-calc" \
        --output-dir "data/sars2_alignment" \
        --sel-key "host: Homo sapiens OR organism: human" \
        --multimer --n-chains 2 \
        --color-seq-match --plot-width 900 \
        --max-mismatches 2 \
        --email "your_email.org" # !! Replace with your email that will be used for Entrez (viral host search)

A fasta file /Users/castelm3/Documents/Github/broad-spectrum-asap-paper/data/sars2_alignment/SARS-CoV-2-Mpro.fasta have been generated with the selected sequences
A fasta file /Users/castelm3/Documents/Github/broad-spectrum-asap-paper/data/sars2_alignment/SARS-CoV-2-Mpro_alignment.fasta have been generated with the multi-seq alignment
A csv file /Users/castelm3/Documents/Github/broad-spectrum-asap-paper/data/sars2_alignment/SARS-CoV-2-Mpro.csv have been generated with the selected sequences
The multi-sequence alignment returns the following matches:
none: 131/320
group: 103/320
exact: 86/320
Aligning 8 sequences of lenght 320
A html file /Users/castelm3/Documents/Github/broad-spectrum-asap-paper/data/sars2_alignment/SARS-CoV-2-Mpro_alignment.html have been generated with the aligned sequences


## 2. ColabFold run to generate AlphaFold2 predicted models

The next step is folding the filtered sequences using the AlphaFold2 multimer model. This step requires a local installation of ColabFold so we provide the output folder in the `data/` directory.

An example script is included below:

```bash
#!/bin/bash
ref_pdb="8dz0.pdb"
csv_fn=SARS-CoV-2-Mpro.csv
template_dir="template_dir_apo"
out_dir="cf_results_human_CoV/

mkdir $template_dir
cp $ref_pdb $template_dir/0001.pdb

module load colabfold/v1.5.2

colabfold_batch $csv_fn $out_dir  \
 --num-recycle 3 --num-models 3 --model-type "alphafold2_multimer_v3" \
 --templates --custom-template-path $template_dir \
```


## 2.1. Structure alignment of ColabFold results

The next step is to extract the structures from the ColabFold run and align them to the reference target. The `asap-spectrum struct-alignment` module takes cares of this (see [documentation](https://asapdiscovery.readthedocs.io/en/latest/index.html) for instructions.)

The command below generates a folder with the aligned ligand-less structures `aligned_structures/`.

In [16]:
!asap-spectrum struct-alignment \
        -f "data/sars2_alignment/SARS-CoV-2-Mpro.csv" \
        --pdb-file {ref_pdb} \
        --output-dir "data/sars2_alignment/aligned_structures/" \
        --cfold-results "data/sars2_alignment/cf_results_human_CoV"\
        --color-by-rmsd \
        --cf-format "alphafold2_multimer_v3" \

RMSD for seed 000 is 2.5291640282355132 A
YP_009725295_1 seed with least RMSD is 000 with RMSD 2.5291640282355132 A
RMSD for seed 000 is 2.7313637564401962 A
YP_009944365_1 seed with least RMSD is 000 with RMSD 2.7313637564401962 A
RMSD for seed 000 is 2.4801465873017134 A
YP_009047217_1 seed with least RMSD is 000 with RMSD 2.4801465873017134 A
RMSD for seed 000 is 2.3460594696310992 A
YP_009944273_1 seed with least RMSD is 000 with RMSD 2.3460594696310992 A
RMSD for seed 000 is 1.9467582184025503 A
YP_009555250_1 seed with least RMSD is 000 with RMSD 1.9467582184025503 A
RMSD for seed 000 is 1.8402437616490752 A
NP_835346_1 seed with least RMSD is 000 with RMSD 1.8402437616490752 A
RMSD for seed 000 is 2.0171923267210268 A
YP_010229075_1 seed with least RMSD is 000 with RMSD 2.0171923267210268 A


## 3. Ligand transfer docking

The next step is to dock the ligand from the reference into the apo proteins. This is done by the `ligand-transfer-docking` command from `asapiscovery`. It takes the folder with aligned structures that we obtained in the previous step as an input, an will return the best pose of each target with the ligans in the reference protein, as well as a summary of the results in a csv file `docking_results_final.csv`. Refer to the `asapdiscovery` documentation for more information.

It is advisable to prepare the proteins before running the docking to cap the termini and fill-in missing side chains. We provided a folder with 4 of the proteins from the previous step, prepared using Maestro. The command below will take a while to run. 

In [ ]:
docking_results_dir = "data/transfer_docking_ensitrelvir"

In [ ]:
!asap-docking ligand-transfer-docking \
        --target SARS-CoV-2-Mpro \
        --structure-dir "data/sars2_alignment/aligned_structures_p/" \
        --ref-pdb-file {ref_pdb} \
        --output-dir {docking_results_dir} \
        --allow-retries --use-omega --omega-dense --allow-final-clash \
        --posit-confidence-cutoff 0.0 \

## 4. MD refinement & Scoring of pose predictions 
Next, we refined the docking poses and score them. This is done by the `score_complexes.py` script included in the `scripts/` folder. The script takes a folder with the posed complexes as input, as well as the reference complex pdb.

From the `ligand-transfer-docking` run above, we need the pdbs of the poses and the results file. We extract the pdb file into a directory `data/docked_ensitrelvir/`, using a bash script (provided in `scripts/`), and then we run the scoring.

In [ ]:
# Extract pdbs from ligand-ransfer-docking directory
protein_regex = "'YP_[0-9]+_[0-9]+|NP_[0-9]+_[0-9]+'" # pattern that matches the name of the targets
docked_dir = "data/docked_ensitrelvir/"

!bash "scripts/copy_posed_pdbs.sh" {docking_results_dir} {docked_dir} {protein_regex}


mkdir: data/docked_ensitrelvir/: File exists
file NP_835346_1
Copied data/transfer_docking_ensitrelvir/docking_results/NP_835346_1_aligned-959df3d38dc535960bdaf7779d68e7117e7abdd77e5fb5edfb40700b81e09d92+QMPBBNUOBOFBFS-LBOYIXSDNA-N+QMPBBNUOBOFBFS-LBOYIXSDNA-N_8dz0_ligand-QMPBBNUOBOFBFS-LBOYIXSDNA-N/docked_complex_0.pdb to data/docked_ensitrelvir//NP_835346_1_docked.pdb
file YP_009047217_1
Copied data/transfer_docking_ensitrelvir/docking_results/YP_009047217_1_aligned-959df3d38dc535960bdaf7779d68e7117e7abdd77e5fb5edfb40700b81e09d92+QMPBBNUOBOFBFS-LBOYIXSDNA-N+QMPBBNUOBOFBFS-LBOYIXSDNA-N_8dz0_ligand-QMPBBNUOBOFBFS-LBOYIXSDNA-N/docked_complex_0.pdb to data/docked_ensitrelvir//YP_009047217_1_docked.pdb
file YP_009725295_1
Copied data/transfer_docking_ensitrelvir/docking_results/YP_009725295_1_aligned-959df3d38dc535960bdaf7779d68e7117e7abdd77e5fb5edfb40700b81e09d92+QMPBBNUOBOFBFS-LBOYIXSDNA-N+QMPBBNUOBOFBFS-LBOYIXSDNA-N_8dz0_ligand-QMPBBNUOBOFBFS-LBOYIXSDNA-N/docked_complex_0.pdb to data/do

In this example run, we will minimize the pdbs to refine the docked poses first, and then we wil do the scoring. The script below will generate a ChemGauss4 score, ML scores (SchNet, E3NN and GAT), Ligand RMSD and AutoDock Vina score. To generate gnina scores the `--gnina-score` command can be used, along with a script to run gnina via  `--gnina-script` and a directory to save the intermediate files in `--home-dir`. A couple of examples gnina scripts are provided in `scripts/`, but a working insatallation of gnina is needed. 

In [ ]:
!python3.10 "scripts/score_complexes.py" \
        -f {ref_pdb} \
        -d {docked_dir} \
        -o "ensitrelvir_scores.csv" \
        --docking-csv {docking_results_dir}/data_intermediates/docking_scores_filtered_sorted.csv \
        --comp-name SARS --target-name SARS-CoV-2-Mpro \
        --protein-regex {protein_regex} \
        --minimize \
        --vina-score --ml-score \
        --path-to-grid-prep "scripts/" \
        --chain-dock "A" --chain-ref "A" \
        --log-level debug \

## Plotting the results